In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.backend import square, mean
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, GRU, Embedding

In [3]:
segment_speeds = pd.read_csv("/home/andrehoejmark/Desktop/GitHub/AVG-Speed-Prediction-of-cars-based-on-road-events/data/segment-data/60-min-intervals/226255131.csv", sep=";")
wind_speed = pd.read_csv("/home/andrehoejmark/Desktop/GitHub/AVG-Speed-Prediction-of-cars-based-on-road-events/data/weather data/smhi-weather-station-data-wind-speed.csv", sep=",")
rain = pd.read_csv("/home/andrehoejmark/Desktop/GitHub/AVG-Speed-Prediction-of-cars-based-on-road-events/data/weather data/smhi-weather-station-data-rain-amount.csv", sep=",")
snow_depth = pd.read_csv("/home/andrehoejmark/Desktop/GitHub/AVG-Speed-Prediction-of-cars-based-on-road-events/data/weather data/smhi-weather-station-data-snow-depth.csv", sep=",")
temperature = pd.read_csv("/home/andrehoejmark/Desktop/GitHub/AVG-Speed-Prediction-of-cars-based-on-road-events/data/weather data/smhi-weather-station-data-temperature.csv", sep=",")

In [4]:
wind_speed = wind_speed.loc[(wind_speed['Datum'] >= "2019-06-04")]
rain = rain.loc[(rain['Datum'] >= "2019-06-04")]
snow_depth = snow_depth.loc[(snow_depth['Datum'] >= "2019-06-04")]
temperature = temperature.loc[(temperature['Datum'] >= "2019-06-04")]

In [5]:
segment_speeds.head(5)

,SegmentId,StartTime,EndTime,Speed
0,226255131,2019-06-03 00:00:00,2019-06-03 01:00:00,NaN
1,226255131,2019-06-03 01:00:00,2019-06-03 02:00:00,NaN
2,226255131,2019-06-03 02:00:00,2019-06-03 03:00:00,NaN
3,226255131,2019-06-03 03:00:00,2019-06-03 04:00:00,NaN
4,226255131,2019-06-03 04:00:00,2019-06-03 05:00:00,NaN


In [6]:
wind_speed.head(5)

,Datum,Tid (UTC),Vindriktning,Kvalitet,Vindhastighet,Kvalitet.1
297120,2019-06-04,00:00:00,192.0,G,4.7,G
297121,2019-06-04,01:00:00,204.0,G,5.0,G
297122,2019-06-04,02:00:00,211.0,G,4.8,G
297123,2019-06-04,03:00:00,214.0,G,4.9,G
297124,2019-06-04,04:00:00,231.0,G,6.7,G


In [7]:
rain.head(5)

,Datum,Tid (UTC),Nederbördsmängd,Kvalitet
206305,2019-06-04,00:00:00,0.0,G
206306,2019-06-04,01:00:00,0.0,G
206307,2019-06-04,02:00:00,0.0,G
206308,2019-06-04,03:00:00,0.0,G
206309,2019-06-04,04:00:00,0.0,G


In [8]:
snow_depth.head(5)

,Datum,Tid (UTC),Snödjup,Kvalitet,Markens tillstånd,Kvalitet.1
4731,2019-06-04,06:00:00,0.0,G,0.0,G
4732,2019-06-05,06:00:00,0.0,G,0.0,G
4733,2019-06-06,06:00:00,0.0,G,0.0,G
4734,2019-06-07,06:00:00,0.0,G,0.0,G
4735,2019-06-08,06:00:00,0.0,G,0.0,G


In [9]:
temperature.head(5)

,Datum,Tid (UTC),Lufttemperatur,Kvalitet
284343,2019-06-04,00:00:00,13.7,G
284344,2019-06-04,01:00:00,14.1,G
284345,2019-06-04,02:00:00,14.2,G
284346,2019-06-04,03:00:00,14.3,G
284347,2019-06-04,04:00:00,13.6,G


# Preprocessing

Removing the first 24 records because the first 24 records are null values for traffic speed

In [10]:
segment_speeds = segment_speeds[24:]

Adding days and hours to the dataset because that could make it easier for it to identify what time instead of receiving a date column which is more complex.

In [11]:
pd.to_datetime(segment_speeds['StartTime'], format="%Y-%m-%d %H:%M:%S")
segment_speeds['hour'] = pd.DatetimeIndex(segment_speeds['StartTime']).hour
segment_speeds['day'] = pd.DatetimeIndex(segment_speeds['StartTime']).dayofyear
segment_speeds['year'] = pd.DatetimeIndex(segment_speeds['StartTime']).year

We now see now many null values are left

In [12]:
pd.isnull(segment_speeds).values.sum()

146

Interpolation is performed for those values that null which means we should have no null values anymore

In [13]:
segment_speeds = segment_speeds.interpolate(method='linear', limit_direction='forward', axis=0)

Now we merge the traffic speeds with the weather data and first need to put the segment speeds in same format as the weather data

In [14]:
pd.to_datetime(temperature['Datum'], format="%Y-%m-%d")
pd.to_datetime(temperature['Tid (UTC)'], format="%H:%M:%S")
temperature['hour'] = pd.DatetimeIndex(temperature['Tid (UTC)']).hour
temperature['day'] = pd.DatetimeIndex(temperature['Datum']).dayofyear
temperature['year'] = pd.DatetimeIndex(temperature['Datum']).year

pd.to_datetime(rain['Datum'], format="%Y-%m-%d")
pd.to_datetime(rain['Tid (UTC)'], format="%H:%M:%S")
rain['hour'] = pd.DatetimeIndex(rain['Tid (UTC)']).hour
rain['day'] = pd.DatetimeIndex(rain['Datum']).dayofyear
rain['year'] = pd.DatetimeIndex(rain['Datum']).year

pd.to_datetime(wind_speed['Datum'], format="%Y-%m-%d")
pd.to_datetime(wind_speed['Tid (UTC)'], format="%H:%M:%S")
wind_speed['hour'] = pd.DatetimeIndex(wind_speed['Tid (UTC)']).hour
wind_speed['day'] = pd.DatetimeIndex(wind_speed['Datum']).dayofyear
wind_speed['year'] = pd.DatetimeIndex(wind_speed['Datum']).year

pd.to_datetime(snow_depth['Datum'], format="%Y-%m-%d")
pd.to_datetime(snow_depth['Tid (UTC)'], format="%H:%M:%S")
snow_depth['hour'] = pd.DatetimeIndex(snow_depth['Tid (UTC)']).hour
snow_depth['day'] = pd.DatetimeIndex(snow_depth['Datum']).dayofyear
snow_depth['year'] = pd.DatetimeIndex(snow_depth['Datum']).year

Verfication that we now see hour, day, year columns with correct values which seem to be correct

In [15]:
temperature.head(2)

,Datum,Tid (UTC),Lufttemperatur,Kvalitet,hour,day,year
284343,2019-06-04,00:00:00,13.7,G,0,155,2019
284344,2019-06-04,01:00:00,14.1,G,1,155,2019


In [16]:
snow_depth.head(2)

,Datum,Tid (UTC),Snödjup,Kvalitet,Markens tillstånd,Kvalitet.1,hour,day,year
4731,2019-06-04,06:00:00,0.0,G,0.0,G,6,155,2019
4732,2019-06-05,06:00:00,0.0,G,0.0,G,6,156,2019


In [17]:
wind_speed.head(2)

,Datum,Tid (UTC),Vindriktning,Kvalitet,Vindhastighet,Kvalitet.1,hour,day,year
297120,2019-06-04,00:00:00,192.0,G,4.7,G,0,155,2019
297121,2019-06-04,01:00:00,204.0,G,5.0,G,1,155,2019


In [18]:
rain.head(2)

,Datum,Tid (UTC),Nederbördsmängd,Kvalitet,hour,day,year
206305,2019-06-04,00:00:00,0.0,G,0,155,2019
206306,2019-06-04,01:00:00,0.0,G,1,155,2019


A left join is done to keep the values from the old data because for example snow_depth only have records certain days and then we could fill the NaN values from the left join with some approximation

In [19]:
segment_speeds_weather = pd.merge(segment_speeds[['SegmentId', 'hour', 'day', 'year', 'Speed']], temperature[['Lufttemperatur', 'hour', 'day', 'year']], on=['day', 'year', 'hour'], how='left')
segment_speeds_weather = pd.merge(segment_speeds_weather, rain[['Nederbördsmängd', 'hour', 'day', 'year']], on=['day', 'year', 'hour'], how='left')
segment_speeds_weather = pd.merge(segment_speeds_weather, wind_speed[['Vindhastighet', 'hour', 'day', 'year']], on=['day', 'year', 'hour'], how='left')
segment_speeds_weather = pd.merge(segment_speeds_weather, snow_depth[['Snödjup', 'hour', 'day', 'year']], on=['day', 'year', 'hour'], how='left')

In [20]:
segment_speeds_weather.head(25)

,SegmentId,hour,day,year,Speed,Lufttemperatur,Nederbördsmängd,Vindhastighet,Snödjup
0,226255131,0,155,2019,91.383333,13.7,0.0,4.7,NaN
1,226255131,1,155,2019,92.000000,14.1,0.0,5.0,NaN
2,226255131,2,155,2019,93.666667,14.2,0.0,4.8,NaN
3,226255131,3,155,2019,86.216667,14.3,0.0,4.9,NaN
4,226255131,4,155,2019,90.116667,13.6,0.0,6.7,NaN
5,226255131,5,155,2019,103.627119,12.9,0.0,6.8,NaN
6,226255131,6,155,2019,92.666667,13.0,0.0,5.2,0.0
7,226255131,7,155,2019,101.366667,13.4,0.0,6.6,NaN
8,226255131,8,155,2019,98.616667,14.1,0.0,6.5,NaN
9,226255131,9,155,2019,93.083333,15.2,0.0,7.3,NaN


In [21]:
segment_speeds_weather.isnull().sum()

SegmentId              0
hour                   0
day                    0
year                   0
Speed                  0
Lufttemperatur      1719
Nederbördsmängd      112
Vindhastighet       1753
Snödjup            20655
dtype: int64

We can now see that there are certain year, day of year and hour combinations that have no values and...

**Lufttemperature(Temperature)** would only miss values if they forgot to take them or temperature insturment stopped working therefore interpolation should work

**Nederbördsmängd** would only miss values if they forgot to take them or temperature insturment stopped working because we can see even if there is no nederbördsmängd they put it at 0 therefore interpolation should work

**Vindhastighet** would only miss values if they forgot to take them or temperature insturment stopped working because it pretty much is always some sort of wind therefore interpolation should work


**snow_depth** gives NaN during the summer months and when there is no snow. Therefore all of the NaN values will be put to 0. It could be that they put 0 during winter months but we just assume that the model would be able to ignore those rare occations

In [22]:
segment_speeds_weather['Lufttemperatur'] = segment_speeds_weather['Lufttemperatur'].interpolate(method='linear', limit_direction='forward', axis=0)
segment_speeds_weather['Nederbördsmängd'] = segment_speeds_weather['Nederbördsmängd'].interpolate(method='linear', limit_direction='forward', axis=0)
segment_speeds_weather['Vindhastighet'] = segment_speeds_weather['Vindhastighet'].interpolate(method='linear', limit_direction='forward', axis=0)
segment_speeds_weather['Snödjup'] = segment_speeds_weather['Snödjup'].fillna(0)

In [23]:
segment_speeds_weather.head(5)

,SegmentId,hour,day,year,Speed,Lufttemperatur,Nederbördsmängd,Vindhastighet,Snödjup
0,226255131,0,155,2019,91.383333,13.7,0.0,4.7,0.0
1,226255131,1,155,2019,92.000000,14.1,0.0,5.0,0.0
2,226255131,2,155,2019,93.666667,14.2,0.0,4.8,0.0
3,226255131,3,155,2019,86.216667,14.3,0.0,4.9,0.0
4,226255131,4,155,2019,90.116667,13.6,0.0,6.7,0.0


Now we can see that all of the NaN values are handled and we should be able to use the data to train

The target that we want to predict

In [24]:
target_names = ['Speed']

In [25]:
# 4 steps of 60min means we try to predict 4 hours into the future
shift_steps_of_60_min = 4

In [26]:
segment_speeds_targets = segment_speeds_weather[['Speed']].shift(-shift_steps_of_60_min)

In [27]:
x_data = segment_speeds_weather[['Speed', 'hour', 'day','Lufttemperatur','Nederbördsmängd','Vindhastighet','Snödjup']].values[0:-shift_steps_of_60_min]

In [28]:
x_data.shape

(21548, 7)

In [29]:
y_data = segment_speeds_targets.values[:-shift_steps_of_60_min]

In [30]:
y_data.shape

(21548, 1)

In [31]:
num_data = len(x_data)
num_data

21548

In [32]:
train_split = 0.9

In [33]:
num_train = int(train_split * num_data)
num_train

19393

In [34]:
#num_test = num_data - num_train
num_test = len(test_segment_speeds)
num_test

NameError: name 'test_segment_speeds' is not defined

In [ ]:
x_train = x_data[0:num_train]
x_test = x_data[num_train:]
len(x_train) + len(x_test)

In [ ]:
y_train = y_data[0:num_train]
y_test = y_data[num_train:]
len(y_train) + len(y_test)

In [ ]:
num_x_signals = x_data.shape[1]
num_x_signals

In [ ]:
num_y_signals = y_data.shape[1]
num_y_signals

In [ ]:
print("Min:", np.min(segment_speeds['Speed']))
print("Max:", np.max(segment_speeds['Speed']))

In [ ]:
x_scaler = MinMaxScaler()

In [ ]:
x_train_scaled = x_scaler.fit_transform(x_train)

In [ ]:
x_test_scaled = x_scaler.transform(x_test)

In [ ]:
print("Min:", np.min(x_train_scaled))
print("Max:", np.max(x_train_scaled))

In [ ]:
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
print(x_train_scaled.shape)
print(y_train_scaled.shape)

In [ ]:
def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

In [ ]:
batch_size = 64

In [ ]:
sequence_length = 24 * 7 * 8
sequence_length

In [ ]:
generator = batch_generator(batch_size=batch_size,
                            sequence_length=sequence_length)

In [ ]:
x_batch, y_batch = next(generator)

In [ ]:
print(x_batch.shape)
print(y_batch.shape)

In [ ]:
batch = 0   # First sequence in the batch.
signal = 0  # First signal from the 20 input-signals.
seq = x_batch[batch, :, signal]
plt.plot(seq)

In [ ]:
validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

# Creation of Model

In [ ]:
model = Sequential()

In [ ]:
model.add(GRU(units=512,
              return_sequences=True,
              input_shape=(None, num_x_signals,)))

In [ ]:
model.add(Dense(num_y_signals, activation='sigmoid'))

In [ ]:
warmup_steps = 50

In [ ]:
def loss_mse_warmup(y_true, y_pred):
    """
    Calculate the Mean Squared Error between y_true and y_pred,
    but ignore the beginning "warmup" part of the sequences.
    
    y_true is the desired output.
    y_pred is the model's output.
    """

    # The shape of both input tensors are:
    # [batch_size, sequence_length, num_y_signals].

    # Ignore the "warmup" parts of the sequences
    # by taking slices of the tensors.
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]

    # These sliced tensors both have this shape:
    # [batch_size, sequence_length - warmup_steps, num_y_signals]

    # Calculat the Mean Squared Error and use it as loss.
    mse = mean(square(y_true_slice - y_pred_slice))
    
    return mse

In [ ]:
optimizer = RMSprop(lr=1e-3)

In [ ]:
model.compile(loss=loss_mse_warmup, optimizer=optimizer)

In [ ]:
model.summary()

In [ ]:
path_checkpoint = '23_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [ ]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=10, verbose=1)

In [ ]:
callback_tensorboard = TensorBoard(log_dir=".\\23_logs\\",
                                   histogram_freq=0,
                                   write_graph=False)

In [ ]:
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-3,
                                       patience=0,
                                       verbose=1)

In [ ]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard,
             callback_reduce_lr]

In [ ]:
model.fit(x=generator,
          epochs=100,
          steps_per_epoch=100,
          validation_data=validation_data,
          callbacks=callbacks)

In [ ]:
try:
    model.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

In [ ]:
result = model.evaluate(x=np.expand_dims(x_test_scaled, axis=0),
                        y=np.expand_dims(y_test_scaled, axis=0))

In [ ]:
print("loss (test-set):", result)

In [ ]:
def plot_comparison(start_idx, length=100, train=True):
    """
    Plot the predicted and true output-signals.
    
    :param start_idx: Start-index for the time-series.
    :param length: Sequence-length to process and plot.
    :param train: Boolean whether to use training- or test-set.
    """
    
    if train:
        # Use training-data.
        x = x_train_scaled
        y_true = y_train
    else:
        # Use test-data.
        x = x_test_scaled
        y_true = y_test
    
    # End-index for the sequences.
    end_idx = start_idx + length
    
    # Select the sequences from the given start-index and
    # of the given length.
    x = x[start_idx:end_idx]
    y_true = y_true[start_idx:end_idx]
    
    # Input-signals for the model.
    x = np.expand_dims(x, axis=0)

    # Use the model to predict the output-signals.
    y_pred = model.predict(x)
    
    # The output of the model is between 0 and 1.
    # Do an inverse map to get it back to the scale
    # of the original data-set.
    y_pred_rescaled = y_scaler.inverse_transform(y_pred[0])
    
    # For each output-signal.
    for signal in range(len(target_names)):
        # Get the output-signal predicted by the model.
        signal_pred = y_pred_rescaled[:, signal]
        
        # Get the true output-signal from the data-set.
        signal_true = y_true[:, signal]

        # Make the plotting-canvas bigger.
        plt.figure(figsize=(15,5))
        
        # Plot and compare the two signals.
        plt.plot(signal_true, label='true')
        plt.plot(signal_pred, label='pred')
        
        # Plot grey box for warmup-period.
        p = plt.axvspan(0, warmup_steps, facecolor='black', alpha=0.15)
        
        # Plot labels etc.
        plt.ylabel(target_names[signal])
        plt.legend()
        plt.show()

In [ ]:
plot_comparison(start_idx=10000, length=1000, train=True)

In [ ]:
plot_comparison(start_idx=0, length=1000, train=False)